In [ ]:
# default_exp ai

# AI - Adding player agents

This is a step towards using this as a game for RL. We define multiple Agents, which follow simple rules in turn.

# Random Agent

TODO description. Trial and error until valid move is found.

In [ ]:
#export
from swoggle.core import *
import random
class RandomAgent:
    """ Given a swoggle board on which it is a player, make a random valid move """
    
    def __init__(self, player):
        self.player = player
    
    def move(self, board, dice_roll):
        
        # If in jail, try to escape
        if self.player in board.jail:
            board.move(self.player, (0, 0), (0, 0), dice_roll, False, False)
            return 'escape'
        
        # Get start_loc
        start_loc = (9, 9)
        for row in board.board:
            for cell in row:
                if cell.player == self.player:
                    start_loc = (cell.y, cell.x)
        if start_loc == (9, 9):
            return None
        
        # Make a random move within reach
        move = ()
        count = 0
        while True:
            count += 1
            end_x = random.choice(range(8))
            end_y = random.choice(range(8))
            drone = random.choice([True, False])
            powerjump = random.choice([True, False])
            valid = board.is_valid_move(self.player, start_loc, (end_x, end_y), dice_roll, drone=drone, powerjump=powerjump)
            move = ()
            if valid:
                move = ((self.player, start_loc, (end_x, end_y), dice_roll, drone, powerjump))
#                 print(f'{self.player} took {count} tries to guess a random move')
                break
        board.move(*move)
        return move
            

### A game with random agents

In [ ]:
sr = Swoggle([RandomAgent(i+1) for i in range(4)])
sr.show()

[1.1][...][...][...][...][...][...][4.4]
[...][...][...][...][...][...][...][...]
[...][...][...][...][.d.][...][...][...]
[...][...][.d.][...][...][...][...][...]
[...][...][...][...][...][.d.][...][...]
[...][...][...][.d.][...][...][...][...]
[...][...][...][...][...][...][...][...]
[2.2][...][...][...][...][...][...][3.3]
Spa: []


In [ ]:
sr.move_agents()
sr.show()

Drone destroyed
Moved 1 (0, 0) (3, 2)
Moved 2 (7, 0) (5, 1)
Moved 3 (7, 7) (7, 5)
Moved 4 (0, 7) (2, 6)
[..1][...][...][...][...][...][...][..4]
[...][...][...][...][...][...][...][...]
[...][...][...][...][.d.][...][4..][...]
[...][...][1..][...][...][...][...][...]
[...][...][...][...][...][.d.][...][...]
[...][2..][...][.d.][...][...][...][...]
[...][...][...][...][...][...][...][...]
[..2][...][...][...][...][3..][...][..3]
Spa: []


# Basic Player

In [ ]:
#export
class BasicAgent:
    """ Given a swoggle board on which it is a player, make a sensible move """
    
    def __init__(self, player):
        self.player = player
    
    def move(self, board, dice_roll):
        
        # If in jail, try to escape
        if self.player in board.jail:
            board.move(self.player, (0, 0), (0, 0), dice_roll, False, False)
            return 'escape'
        
        # Get start_loc
        start_loc = (9, 9)
        for row in board.board:
            for cell in row:
                if cell.player == self.player:
                    start_loc = (cell.y, cell.x)
        if start_loc == (9, 9):
            return None
        
        
        # If bases in range, take them
        
        for row in board.board:
            for cell in row:
                if cell.player == None and cell.base != None and cell.base != self.player: # Normal move
                    move = (self.player, start_loc, (cell.x, cell.y), dice_roll, False, False)
                    if board.is_valid_move(*move):
                        board.move(*move)
                        return (move)
                    
                if cell.base != None and cell.base != self.player: # Drone attack
                    move = (self.player, start_loc, (cell.x, cell.y), dice_roll, True, False)
                    if board.is_valid_move(*move):
                        board.move(*move)
                        return (move)
        
        # If on base and player in range, take or powerjump them
        if board.board[start_loc[0]][start_loc[1]].base == self.player:
            for row in board.board:
                for cell in row:
                    if cell.player != None and cell.player != self.player:
                        # try normal move
                        move = (self.player, start_loc, (cell.x, cell.y), dice_roll, False, False)
                        if board.is_valid_move(*move):
                            board.move(*move)
                            return (move)
                        # Try powerjump
                        move = (self.player, start_loc, (cell.x, cell.y), dice_roll, False, True)
                        if board.is_valid_move(*move):
                            board.move(*move)
                            return (move)
        
        # If players in range and takeable, take them
        for row in board.board:
            for cell in row:
                if cell.player != None and cell.player != self.player:
                    # Normal take
                    move = (self.player, start_loc, (cell.x, cell.y), dice_roll, False, False)
                    if board.is_valid_move(*move):
                        board.move(*move)
                        return (move)
                    # Drone take
                    move = (self.player, start_loc, (cell.x, cell.y), dice_roll, True, False)
                    if board.is_valid_move(*move):
                        board.move(*move)
                        return (move)
        
        # TODO: If player close to your base and base reacheable, go back to base
        
        # Else move randomly
        
        # Make a random move within reach
        move = ()
        count = 0
        while True:
            count += 1
            end_x = random.choice(range(8))
            end_y = random.choice(range(8))
            drone = random.choice([True, False])
            powerjump = random.choice([True, False])
            valid = board.is_valid_move(self.player, start_loc, (end_x, end_y), dice_roll, drone=drone, powerjump=powerjump)
            move = ()
            if valid:
                move = ((self.player, start_loc, (end_x, end_y), dice_roll, drone, powerjump))
#                 print(f'{self.player} took {count} tries to guess a random move')
                break
        board.move(*move)
        return move

### Here's a way to get win rates for different agents over n games:

In [ ]:
#export
from IPython.display import clear_output

def win_rates(n, agents):
    wins = {}
    for i in range(n):
        rounds = 0
        sr = Swoggle(agents, verbose=False)
        while True:
            sr.move_agents()
            rounds += 1
            players = []
            for row in sr.board.board:
                for cell in row:
                    if cell.player != None:
                        players.append(cell.player)
            if len(players) <= 1:
                clear_output(wait=True)
                print("Winner:", players, rounds)
                if len(players) == 1:
                    if players[0] in wins:
                        wins[players[0]] += 1
                    else:
                        wins[players[0]] = 1
                break
    return wins




Winner: [3] 67
{2: 116, 4: 315, 3: 62, 1: 7}


Interestingly, the one opposite the non-random player (player 2) also gets some advantage - the nearby opponents are often removed. 

In [ ]:
print(win_rates(500, [RandomAgent(i+1) for i in range(3)]+[BasicAgent(4)]))